In [1]:
import pybithumb
import numpy as np

def get_ror(k = 0.5, fee=0.0032):
    df = pybithumb.get_ohlcv("BTC")
    df['range'] = (df['high'] - df['low']) * k
    df['range_shift1'] = df['range'].shift(1)
    df['target'] = df['open'] + df['range_shift1']

    df['ror'] = np.where(df['high'] > df['target'], 
                         df['close'] / df['target'] -fee,
                         1)
    ror = df['ror'].cumprod()[-2]
    return ror

for k in np.arange(0.1, 1.0, 0.1):
    ror = get_ror(k)
    print("%.1f %f" % (k, ror))


df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['range_shift1'] = df['range'].shift(1)
df['target'] = df['open'] + df['range_shift1']

df['ror'] = np.where(df['high'] > df['target'], 
                     df['close'] / df['target'] -0.0032,
                     1)

ror = df['ror'].cumprod()[-2]
df['hpr'] = df['ror'].cumprod()
print(df.tail())
df.to_excel("btc.xlsx")

0.1 0.008529
0.2 0.048746
0.3 0.167791
0.4 1.030977
0.5 2.144039
0.6 4.924765
0.7 5.718025
0.8 5.969936
0.9 4.004160
                           open        high         low       close  \
time                                                                  
2021-04-09 00:00:00  74313000.0  75986000.0  73154000.0  75471000.0   
2021-04-10 00:00:00  75443000.0  79154000.0  75000000.0  78514000.0   
2021-04-11 00:00:00  78480000.0  79100000.0  76300000.0  77688000.0   
2021-04-12 00:00:00  77646000.0  78900000.0  77281000.0  78106000.0   
2021-04-13 07:00:00  78119000.0  78400000.0  77770000.0  78168000.0   

                          volume      range  range_shift1      target  \
time                                                                    
2021-04-09 00:00:00  4106.284156  1416000.0     2139000.0  76452000.0   
2021-04-10 00:00:00  3855.943855  2077000.0     1416000.0  76859000.0   
2021-04-11 00:00:00  3713.849876  1400000.0     2077000.0  80557000.0   
2021-04-12 00:00:00 

In [3]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")

df['ma5'] = df['close'].rolling(window=5).mean().shift(1)
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)
df['bull'] = df['open'] > df['ma5']

fee = 0.0032
df['ror'] = np.where((df['high'] > df['target']) & df['bull'],
                  df['close'] / df['target'] - fee,
                  1)

df['hpr'] = df['ror'].cumprod()
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100
print("MDD: ", df['dd'].max())
print("HPR: ", df['hpr'][-2])
df.to_excel("larry_ma.xlsx")

MDD:  41.07120419932468
HPR:  17.33278445939211


In [4]:
import pybithumb
import numpy as np


def get_hpr(ticker):
    try:
        df = pybithumb.get_ohlcv(ticker)
        df = df['2018']

        df['ma5'] = df['close'].rolling(window=5).mean().shift(1)
        df['range'] = (df['high'] - df['low']) * 0.5
        df['target'] = df['open'] + df['range'].shift(1)
        df['bull'] = df['open'] > df['ma5']

        fee = 0.0032
        df['ror'] = np.where((df['high'] > df['target']) & df['bull'],
                              df['close'] / df['target'] - fee,
                              1)

        df['hpr'] = df['ror'].cumprod()
        df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100
        return df['hpr'][-2]
    except:
        return 1


tickers = pybithumb.get_tickers()

hprs = []
for ticker in tickers:
    hpr = get_hpr(ticker)
    hprs.append((ticker, hpr))

sorted_hprs = sorted(hprs, key=lambda x:x[1], reverse=True)
print(sorted_hprs[:5])

[('REP', 8.40193145161698), ('POWR', 5.65256529365624), ('SNT', 4.0216481243550755), ('LRC', 2.9260451669193657), ('XRP', 2.469533973902334)]
